# Script to add bond additivity corrections (BAC) for new level of theory in Arkane
## Do atomization energies (AE) first!
See documentation for current levels of theory: https://github.com/ReactionMechanismGenerator/RMG-Py/blob/2a1f9100390fc2ef1b8a8102925d06b941695a25/documentation/source/users/arkane/input.rst#L57

This paper explains atomization energy corrections and bond additivity corrections:
https://onlinelibrary.wiley.com/doi/10.1002/kin.21637


# Step 1: Load reference database

In [2]:
import os
# import glob

import ase
import ase.io.gaussian

import rmgpy.species

import arkane.encorr.data as data
import arkane.encorr.reference
import arkane.modelchem

import arkane.encorr.ae
import arkane.ess.gaussian




/home/harris.se/rmg/RMG-Py/rmgpy/rmg/reactors.py:53: RuntimeWarning: Unable to import Julia dependencies, original error: [Errno 2] No such file or directory: 'julia': 'julia'
  warnings.warn("Unable to import Julia dependencies, original error: " + str(e), RuntimeWarning)


In [3]:
ref_db = arkane.encorr.reference.ReferenceDatabase()
ref_db.load()


*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3);
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3);
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual v

*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(2)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(2)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O+1(2)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O+1(2)
ERROR:root:Unable to generate identifier for this molecule:
1 O u0 p3 c-1 {3,S}
2 O u0 p2 c0 {3,D}
3 C u0 p0 c0 {1,S} {2,D} {4,S}
4 H u0 p0 c0 {3,S}

*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valen

*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): S(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): S(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): S(1); O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): S(1); O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): C(3)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted unusual valence(s): O(1)
*** Open Babel Warning  in InChI code
  #1 :Accepted

In [ ]:
# we'll add to this with
update_from_arkane_spcs(self, arkane_species)

# Step 2: Set up DFT calculations using previously calculated geometries as starting point

In [4]:
start_dir = '/home/harris.se/rmg/rmg_tools/arkane/new_lot'
working_dir = os.path.join(start_dir, 'bac_calcs', 'm062x_ccpvtz')
# working_dir = os.path.join(start_dir, 'bac_calcs', 'b3lyp_6311G3df2p')
# working_dir = os.path.join(start_dir, 'bac_calcs', 'ccsdt_ccpvtz')

os.makedirs(working_dir, exist_ok=True)

LevelOfTheory(method='wb97xd3',basis='def2tzvp',software='qchem')

In [31]:
# adjust these as needed
method = 'm062x'
basis = 'cc-pvtz'

lot = arkane.modelchem.LevelOfTheory(
    method=method,
    basis=basis,
    software='gaussian'
)

for i in range(len(ref_db.reference_sets['main'])):
    
    s = rmgpy.species.Species().from_adjacency_list(ref_db.reference_sets['main'][i].adjacency_list)
    
    # get starting geometry from previous calculations
    preferred_methods = ['ccsd(t)f12', 'cbsqb32023']
    atoms = None
    for preferred_method in preferred_methods:
        for key in ref_db.reference_sets['main'][i].calculated_data.keys():
            comparison = key
            if type(key) != arkane.modelchem.LevelOfTheory:
                comparison = key.energy
            
            if comparison.method == preferred_method:
                syms = ref_db.reference_sets['main'][i].calculated_data[key].xyz_dict['symbols']
                xyz = ref_db.reference_sets['main'][i].calculated_data[key].xyz_dict['coords']
                atoms = ase.Atoms(symbols=syms, positions=xyz)
                break
        if atoms:
            break
    else:
        raise ValueError(f'no preferred level of theory for entry {i}')


    sp_dir = os.path.join(working_dir, f'species_{i:04}')
    os.makedirs(sp_dir, exist_ok=True)
    
    
    radical_electrons = any([x.radical_electrons for x in s.molecule[0].atoms])
    
    # specific modification for CCSD(T)
    specific_method = lot.method
    if radical_electrons and lot.method == 'ccsd(t)':
        specific_method = 'uccsd(t)'
    
    # write the gaussian calculation file
    with open(os.path.join(sp_dir, 'sp.com'), 'w') as f:
        ase.io.gaussian.write_gaussian_in(
            f,
            atoms,
            properties=['energy'],
            method=specific_method,
            basis=basis,
            mult=s.multiplicity,
            opt='calcfc,maxcycles=900'
        )

# write the slurm script
run_script = os.path.join(working_dir, 'run.sh')
with open(run_script, 'w') as f:
    f.write("""#!/bin/bash
#SBATCH --job-name=g16_bac_opt
#SBATCH --error=error.log
#SBATCH --nodes=1
#SBATCH --partition=west,short
#SBATCH --exclude=c5003
#SBATCH --mem=20Gb
#SBATCH --time=24:00:00
#SBATCH --cpus-per-task=16
""" +
f'#SBATCH --array=0-{len(ref_db.reference_sets["main"]) - 1}%30\n' +
"""
export GAUSS_SCRDIR=/scratch/harris.se/guassian_scratch
mkdir -p $GAUSS_SCRDIR
module load gaussian/g16
source /shared/centos7/gaussian/g16/bsd/g16.profile

RUN_i=$(printf "%04.0f" $(($SLURM_ARRAY_TASK_ID)))

cd "species_${RUN_i}"
g16 sp.com

""")


In [32]:
working_dir

'/home/harris.se/rmg/rmg_tools/arkane/new_lot/bac_calcs/ccsdt_ccpvtz'

# Step 3 - run geometry optimizations

In [34]:
os.chdir(working_dir)
os.system(f'chmod +x {run_script}') # <-- might need to change file permissions
os.system(f'sbatch {run_script}')

Submitted batch job 45530790


0

# Step 4 - grab the energies for the species_energies dictionary

In [ ]:
species_energies = {}
for i in sp_indices:
    # get the gaussian logfile
    logfile = os.path.join(working_dir, f'species_{i:04}', 'sp.log')
    
    with open(logfile, 'r') as f:
        gaussian_log = arkane.ess.gaussian.GaussianLog(logfile)
        # Notes from arkane.encorr.ae say ZPE should not be included
        # arkane.ess.gaussian comments say ZPE is not included
        energy = gaussian_log.load_energy() / 2625500  # convert to Hartrees
        species_energies[ref_db.reference_sets['main'][i].label] = energy
        

In [ ]:
species_energies

In [ ]:
# write the arkane ae job file (or I could do this in the code...)
ae_jobfile = os.path.join(working_dir, 'input.py')


with open(ae_jobfile, 'w') as f:
    f.write("""title = 'Atom energy correction fitting for """ + f'{lot.method}/{lot.basis} ' + """level of theory'

lot = LevelOfTheory(\n""" +
f"    method='{lot.method}',\n" +
f"    basis='{lot.basis}',\n" +
"""    software=gaussian
)

""" + 

f"species_energies={species_energies}\n" + 

"""
ae(
    species_energies=species_energies,
    level_of_theory=lot,
    write_to_database=False,
    overwrite=False
)

""")




In [ ]:
# same thing as the job above, but in code
ae_job = arkane.encorr.ae.AEJob(
    species_energies=species_energies,
    level_of_theory=lot,
    write_to_database=False,
    overwrite=False
)
ae_job.execute()

In [ ]:
ae_job.ae.atom_energies

In [ ]:
ae_job.ae.confidence_intervals

In [ ]:
ae_job.ae.atom_energies.keys()

In [ ]:
# Copy formatted energies into 

In [ ]:
# add these to the atom_energies dictionary in RMG-database/input/quantum_corrections/data.py

for atom in ae_job.ae.atom_energies.keys():
    print(f'# {atom}\t{ae_job.ae.atom_energies[atom]:.8f}\t+/- {ae_job.ae.confidence_intervals[atom]:.8f} Hartree')

print(f'"{lot} ":','{')


for atom in ae_job.ae.atom_energies.keys():
    print(f'    "{atom}": {ae_job.ae.atom_energies[atom]},')

print('}')
# ae_job.ae.atom_energies

